In [1]:
import tweepy

bearer_token = "AAAAAAAAAAAAAAAAAAAAAAhrjgEAAAAAtaINtJuJDU%2F%2BCdOGL3BFon0rYzo%3DRBGMBkpmeImGMYSEnM0Sa8F2mCwZlidarulMjtMOuKqla8aWCs" # Twitter API
client = tweepy.Client(bearer_token=bearer_token) # Twitter API v2 with OAuth2.0 Version

#let's request and paginate the tweets
query = 'context:152.825047692124442624 -is:retweet lang:en'

limit = 1000 #n of tweets to be requested


tweets = tweepy.Paginator(client.search_recent_tweets,
                          query=query,
                          expansions=['author_id'],
                          tweet_fields=['public_metrics'],
                          max_results=100).flatten(limit=limit)

In [2]:
#Let's create the panda dataframe
import pandas as pd

df_tweets = pd.DataFrame([tweet.data['text'],
                          tweet.data['public_metrics']['retweet_count'],
                          tweet.data['public_metrics']['like_count'],
                          tweet.data['author_id']
                         ] for tweet in tweets) # convert to pandas df

#Let's rename the columns
df_tweets.rename(columns={0: "Text", 1: "Retweet", 2: "Likes", 3: "user_id"}, inplace=True)

display(df_tweets)

,Text,Retweet,Likes,user_id
0,Were you playing football today? You smell lik...,0,0,1487514194933161989
1,Shrimp top ramen hit everytime,0,0,556611097
2,Korean food quiz!! 👀\nDo you know what this fo...,0,0,1572469257522515970
3,"Made...no hay rips ""His"" lunch,... Criticized\...",0,0,2266985348
4,@FightHaven The guy ate his lunch😭😭😭,0,0,1018910081683947520
...,...,...,...,...
995,The Mediterranean Diet Cookbook for Beginners:...,0,0,1584767933808517121
996,@Jacob_Rees_Mogg You couldn't organise a piss ...,0,0,1567957321279488010
997,"Ultimate Breakfast Cookbook: Eggs, Pancakes, C...",0,0,1586672083597234177
998,Hub:Pooja My Boss Coming To Our Home for Lunch...,3,15,1026531765408681984


In [3]:
#getting the number of followers(we need to do a for loop because we can ask for up to 100 followers at the time)
user_list = list(df_tweets["user_id"])
followers = pd.DataFrame()
#print(user_list[(1-1)*100:1*100-1])
for i in range(0,limit//100):
    users = tweepy.Paginator(client.get_users, 
                                 ids=user_list[i*100:(i+1)*100],
                                 user_fields=['public_metrics']).flatten(limit=100)
    f = pd.DataFrame([user.data['public_metrics']['followers_count'] for user in users])
    followers = pd.concat([followers,f])
    
followers = followers.reset_index(drop=True)

#Let's add the column for the followers
df_tweets['followers'] = followers
display(df_tweets)

Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


,Text,Retweet,Likes,user_id,followers
0,Were you playing football today? You smell lik...,0,0,1487514194933161989,12
1,Shrimp top ramen hit everytime,0,0,556611097,11261
2,Korean food quiz!! 👀\nDo you know what this fo...,0,0,1572469257522515970,123
3,"Made...no hay rips ""His"" lunch,... Criticized\...",0,0,2266985348,0
4,@FightHaven The guy ate his lunch😭😭😭,0,0,1018910081683947520,198
...,...,...,...,...,...
995,The Mediterranean Diet Cookbook for Beginners:...,0,0,1584767933808517121,6
996,@Jacob_Rees_Mogg You couldn't organise a piss ...,0,0,1567957321279488010,320
997,"Ultimate Breakfast Cookbook: Eggs, Pancakes, C...",0,0,1586672083597234177,4
998,Hub:Pooja My Boss Coming To Our Home for Lunch...,3,15,1026531765408681984,49224


In [4]:
'''
#getting the number of followers(we need to do a for loop because we can ask for up to 100 followers at the time)
user_list = list(df_tweets["user_id"])
followers = []
for i in range(1,limit//100):
    users = tweepy.Paginator(client.get_users, 
                                 ids=user_list[(i-1)*100:i*100],
                                 user_fields=['public_metrics']).flatten(limit=100)
    f = [user.data['public_metrics']['followers_count'] for user in users]
    followers.append(f)
    
print(followers)
    
#Let's add the column for the followers
#df_tweets['followers'] = followers
'''

'\n#getting the number of followers(we need to do a for loop because we can ask for up to 100 followers at the time)\nuser_list = list(df_tweets["user_id"])\nfollowers = []\nfor i in range(1,limit//100):\n    users = tweepy.Paginator(client.get_users, \n                                 ids=user_list[(i-1)*100:i*100],\n                                 user_fields=[\'public_metrics\']).flatten(limit=100)\n    f = [user.data[\'public_metrics\'][\'followers_count\'] for user in users]\n    followers.append(f)\n    \nprint(followers)\n    \n#Let\'s add the column for the followers\n#df_tweets[\'followers\'] = followers\n'

In [5]:
import os

path = os.getcwd()
df_tweets.to_csv(os.path.join(path,'Dataset1.csv'))#creates a csv file containing the tweets